# University of Sheffield - School of Mathematics and Statistics
## MAS316/MAS414/MAS6446: Mathematical Modelling of Natural Systems
João Carreiras - G39e - j.carreiras@sheffield.ac.uk
### Topic III: Machine learning methods to retrieve forest biophysical parameters
### Practical #1: Predicting the age of secondary forests with decision trees, bagging and boosting methods 

### <span style="color:red">*Confidentiality: The contents of this Jupyter Notebook are not to be shared outside this course*</span>

### Context and objectives
In the tropics, deforestation typically results in the replacement of old-growth forests by croplands and pastures but these are often abandoned after a few years and replaced with naturally regrowing forests - secondary forests. These secondary forests have different ages, and accumulate aboveground biomass and restore biodiversity lost previously during the initial deforestation process.

In this practical session we'll be using a dataset containing the age of secondary forests in the Amazon. The objective is to test several machine learning algorithms (decision trees, bagging, boosting) to estimate the age of secondary forests with several predictors obtained from satellite observations. We will assess the performance of each algorithm by computing the error obtained with an independent sample.

### Data

Landsat 5 Thematic Mapper (TM) is a satellite sensor that acquired observations over the Earth surface between 1984 and 2013. The Landsat 5 satellite orbited the the Earth in a sun-synchronous, near-polar orbit, at an altitude of 705 km (438 mi), inclined at 98.2 degrees, and circled the Earth every 99 minutes. The TM sensor onboard Landsat 5 is an optical sensor, acquiring observations in seven spectral bands:
* Band 1: visible (0.45 - 0.52 µm)
* Band 2: visible (0.52 - 0.60 µm)
* Band 3: visible (0.63 - 0.69 µm)
* Band 4: near-infrared (0.76 - 0.90 µm)
* Band 5: near-infrared (1.55 - 1.75 µm)
* Band 6: thermal (10.40 - 12.50 µm) - we won't be using this band
* Band 7: mid-infrared (2.08 - 2.35 µm)

Advanced Land Observing Satellite (ALOS) Phased Arrayed L-band Synthetic Aperture Radar (PALSAR) yielded detailed, all-weather, day-and-night observation of the Earth's surface between 2006 and 2011. Data acquired by the PALSAR sensor are from multiple observation modes with variable polarization, resolution, swath width, and off-nadir angle. In this exercise we will use the following variables:
* HH: backscatter intensity in the HH polarisation
* HV: backscatter intensity in the HV polarisation

The dataset has 1,876 observations of the age of secondary forests and eight predictors. The predictors were obtained from ALOS PALSAR and Landsat 5 TM; they are all adimensional.

* *ID*: code assigned to each observation
* *ASF*: age of secondary forests (years)
* *HH*: ALOS PALSAR backscatter intensity - HH polarisation
* *HV*: ALOS PALSAR backscatter intensity - HV polarisation
* *b1*: Landsat 5 TM surface reflectance - Band 1
* *b2*: Landsat 5 TM surface reflectance - Band 2
* *b3*: Landsat 5 TM surface reflectance - Band 3
* *b4*: Landsat 5 TM surface reflectance - Band 4
* *b5*: Landsat 5 TM surface reflectance - Band 5
* *b7*: Landsat 5 TM surface reflectance - Band 7

### Reading the data
Throughout this practical session we will use several Python libraries: pandas, numpy, mathplotlib, sklearn; [sklearn](https://scikit-learn.org/stable) offers numerous tools for predictive data analysis, hosting implementations of the most important machine learning tools we'll be using during this topic.

In [ ]:
import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics
from sklearn import tree
from sklearn.inspection import permutation_importance

import matplotlib.pyplot as plt

from tabulate import tabulate

We'll first read the dataset containing the observations from a csv file on a shared google drive. Those not being able to access google, please download the offline file in Blackboard (asf_amazon.csv), save it to your computer, and change the path in the code below to have it read into the pandas dataframe.

In [ ]:
asf_amazon = pd.read_csv("https://drive.google.com/uc?export=download&id=1gYcYOTEHr7d-zh9q-H2i-Gy0leuQUMf3")

In [ ]:
## Check descriptive statistics
asf_amazon.describe()

### Response, predictors and partitioning the dataset into training and testing subsets
In this exercise we will test several machine learning algorithms to estimate the age of secondary forests as a function of several measurements:

*ASF* = f(*HH*, *HV*, *b1*, *b2*, *b3*, *b4*, *b5*, *b7*)

We will use a 75:25 random partition of the dataset to obtain the training and testing subsets, respectively. The training subset will be used for model fitting, whereas the testing subset will be kept aside and only used when assessing the predictive capability of the models. <span style="color:red">Please note, we need to set a random number so that we can reproduce the results</span>.

In [ ]:
y = asf_amazon.ASF ## set the response
X = asf_amazon[['HH','HV','b1','b2','b3','b4','b5','b7']] ## set the predictors
## Split into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234)

### Fitting regression trees
The scikit-learn library has a specific function to fit a regression tree based on the CART algorithm (see Lecture Notes). The function is called [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor), and in the link you can learn about the parameters controlling this function.

In the code below, we'll specify the cost-complexity pruning approach to select the best tree.

In [ ]:
## Specifying the full tree (according to defaults)
dtr_full = DecisionTreeRegressor(random_state=1234)
##
## Fitting the full tree
dtr_full.fit(X_train, y_train)
##
## Plot the tree
tree.plot_tree(dtr_full, feature_names=X_train.columns)
plt.show()

In [ ]:
## Predicting to the train and test subsets
y_train_pred_full = dtr_full.predict(X_train)
y_test_pred_full  = dtr_full.predict(X_test)
##
## bias-variance decomposition of the error
dtr_full_train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred_full))
dtr_full_train_bias = np.mean(y_train - y_train_pred_full)
dtr_full_train_var  = np.var(y_train - y_train_pred_full)
print(dtr_full_train_rmse, dtr_full_train_bias, dtr_full_train_var)
dtr_full_test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred_full))
dtr_full_test_bias = np.mean(y_test - y_test_pred_full)
dtr_full_test_var  = np.var(y_test - y_test_pred_full)
print(dtr_full_test_rmse, dtr_full_test_bias, dtr_full_test_var)
##
## Can you comment on the results?
## The full tree perfectly fits the data - overfitting
## BUT when applied to the test subset, the errors are quite high
##
## What's the RMSE in relation to the average ASF value in the test subset?
av_asf_test = np.mean(y_test)
print(dtr_full_test_rmse/av_asf_test*100)

In [ ]:
## Scikit-learn has a function called cost_complexity_pruning_path, which gives the effective alpha
## of subtrees during pruning and the corresponding impurities (see Lecture Notes - Equation 1.9). 
## We can use these values of alpha to prune our regression tree
path = dtr_full.cost_complexity_pruning_path(X_train, y_train)
alpha = path.ccp_alphas

In [ ]:
## We will use these values of alpha and pass it to the ccp_alpha parameter of our DecisionTreeRegressor
## By looping over the alphas array, we will find the error on the train and test subsets
rmse_train,rmse_test = [],[]
for i in alpha:
    dtr = DecisionTreeRegressor(ccp_alpha=i, random_state=1234)
    dtr.fit(X_train, y_train)
    y_train_pred = dtr.predict(X_train)
    y_test_pred  = dtr.predict(X_test)
    rmse_train.append(np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
    rmse_test.append(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

In [ ]:
## Plotting alpha vs RMSE (train and test subsets)
fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("RMSE (years)")
ax.plot(alpha, rmse_train, label="train")
ax.plot(alpha, rmse_test, label="test")
plt.xlim([0, 0.7])
ax.legend()
plt.show()

In [ ]:
## Let's choose the alpha corresponding to the "best" model, i.e., that with the lowest test RMSE
index_best_model = np.argmin(rmse_test)
alpha_best = alpha[index_best_model]
print(alpha_best)

In [ ]:
## Let's then fit a simpler tree using alpha=alpha_best
dtr_alpha = DecisionTreeRegressor(ccp_alpha=alpha_best, random_state=1234)
dtr_alpha.fit(X_train, y_train)
y_test_pred_alpha = dtr_alpha.predict(X_test)

In [ ]:
## Plotting the observed vs predicted tree aboveground biomass
plt.scatter(y_test, y_test_pred_alpha, s=60, edgecolor='black', c='red', label='data, alpha_best')
plt.plot(y_test, y_test, c='blue', label='y=x')
plt.xlim([0, 30])
plt.ylim([0, 30])
plt.xlabel("observed age of secondary forests (years)")
plt.ylabel("predicted age of secondary forests (years)")
plt.legend()
plt.show()

In [ ]:
## Now, let's compare with the full tree
## Plotting the observed vs predicted tree aboveground biomass
plt.scatter(y_test, y_test_pred_full, s=60, edgecolor='black', c='red', label='data, full tree')
plt.plot(y_test, y_test, c='blue', label='y=x')
plt.xlim([0, 30])
plt.ylim([0, 30])
plt.xlabel("observed age of secondary forests (years)")
plt.ylabel("predicted age of secondary forests (years)")
plt.legend()
plt.show()

#### Calculating the bias-variance decomposition of the test error
Use Equation 1.19 to calculate the bias-variance decomposition of the root mean square error.

In [ ]:
dtr_alpha_rmse = "{:.4f}".format(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred_alpha)))
dtr_alpha_bias = "{:.4f}".format(np.mean(y_test - y_test_pred_alpha))
dtr_alpha_var  = "{:.4f}".format(np.var(y_test - y_test_pred_alpha))
print(dtr_alpha_rmse, dtr_alpha_bias, dtr_alpha_var)

In [ ]:
dtr_full_rmse = "{:.4f}".format(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred_full)))
dtr_full_bias = "{:.4f}".format(np.mean(y_test - y_test_pred_full))
dtr_full_var  = "{:.4f}".format(np.var(y_test - y_test_pred_full))
print(dtr_full_rmse, dtr_full_bias, dtr_full_var)

#### Calculating the importance of each predictor
We can use the function permutation_importance to rank the predictors being used to fit any given model. This will give us some insight about the variables that are more important to estimate the aboveground biomass of each tree.

In [ ]:
dtr_alpha_imp = permutation_importance(dtr_alpha, X_train, y_train, n_repeats=100, random_state=1234)
sort_imp = dtr_alpha_imp.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(dtr_alpha_imp.importances[sort_imp].T,
           vert=False, labels=X_train.columns[sort_imp])
ax.set_title("Permutation Importances (train subset)")
fig.tight_layout()
plt.show()

In [ ]:
dtr_full_imp = permutation_importance(dtr_full, X_train, y_train, n_repeats=100, random_state=1234)
sort_imp = dtr_full_imp.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(dtr_full_imp.importances[sort_imp].T,
           vert=False, labels=X_train.columns[sort_imp])
ax.set_title("Permutation Importances (train subset)")
fig.tight_layout()
plt.show()

### Fitting bagging models
The scikit-learn library has a specific function to fit a bagging regression model (see Lecture Notes). The function is called [BaggingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html#sklearn.ensemble.BaggingRegressor), and in the link you can learn about the parameters controlling this function.

In the code below you'll have to specify two bagging models, using a regression tree as the weak learner :

1. Setting 100 estimators, i.e., 100 regression trees
2. Setting 600 estimators, i.e., 600 regression trees

You can leave the other parameters as default.

In [ ]:
## Specifying the bagging models
bagg_1 = BaggingRegressor(DecisionTreeRegressor(), n_estimators=100, random_state=1234)
bagg_2 = BaggingRegressor(DecisionTreeRegressor(), n_estimators=600, random_state=1234)
##
## Fitting the bagging models
bagg_1.fit(X_train, y_train)
bagg_2.fit(X_train, y_train)
##
## Predicting to the test subset
bagg_1.pred = bagg_1.predict(X_test)
bagg_2.pred = bagg_2.predict(X_test)
##
## Estimating the bias, variance and root mean squared error
bagg_1_rmse = "{:.4f}".format(np.sqrt(metrics.mean_squared_error(y_test, bagg_1.pred)))
bagg_1_bias = "{:.4f}".format(np.mean(y_test - bagg_1.pred))
bagg_1_var  = "{:.4f}".format(np.var(y_test - bagg_1.pred))
print(bagg_1_rmse, bagg_1_bias, bagg_1_var)
bagg_2_rmse = "{:.4f}".format(np.sqrt(metrics.mean_squared_error(y_test, bagg_2.pred)))
bagg_2_bias = "{:.4f}".format(np.mean(y_test - bagg_2.pred))
bagg_2_var  = "{:.4f}".format(np.var(y_test - bagg_2.pred))
print(bagg_2_rmse, bagg_2_bias, bagg_2_var)

In [ ]:
## Plotting the observed vs predicted tree aboveground biomass
plt.scatter(y_test, bagg_1.pred, s=60, edgecolor='black', c='red', label='data, 100 estimators')
plt.plot(y_test, y_test, c='blue', label='y=x')
plt.xlim([0, 30])
plt.ylim([0, 30])
plt.xlabel("observed age of secondary forests (years)")
plt.ylabel("predicted age of secondary forests (years)")
plt.legend()
plt.show()

In [ ]:
## Plotting the observed vs predicted tree aboveground biomass
plt.scatter(y_test, bagg_2.pred, s=60, edgecolor='black', c='red', label='data, 600 estimators')
plt.plot(y_test, y_test, c='blue', label='y=x')
plt.xlim([0, 30])
plt.ylim([0, 30])
plt.xlabel("observed age of secondary forests (years)")
plt.ylabel("predicted age of secondary forests (years)")
plt.legend()
plt.show()

In [ ]:
## Calculate variable importance
bagg_1_imp = permutation_importance(bagg_1, X_train, y_train, n_repeats=100, random_state=1234)
sort_imp = bagg_1_imp.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(bagg_1_imp.importances[sort_imp].T,
           vert=False, labels=X_train.columns[sort_imp])
ax.set_title("Permutation Importances (train subset)")
fig.tight_layout()
plt.show()

In [ ]:
## Calculate variable importance
bagg_2_imp = permutation_importance(bagg_2, X_train, y_train, n_repeats=100, random_state=1234)
sort_imp = bagg_2_imp.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(bagg_2_imp.importances[sort_imp].T,
           vert=False, labels=X_train.columns[sort_imp])
ax.set_title("Permutation Importances (train subset)")
fig.tight_layout()
plt.show()

### Fitting boosting models
The scikit-learn library has a specific function to fit an AdaBoost regression model (see Lecture Notes). The function is called [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor), and in the link you can learn about the parameters controlling this function.

In the code below you'll have to specify two boosting models, using a regression tree as the weak learner :

1. Setting 100 estimators, i.e., 100 regression trees, learning rate = 0.1
2. Setting 600 estimators, i.e., 600 regression trees, learning rate = 0.01

You can leave the other parameters as default.

In [ ]:
## Specifying the boosting model
boost_1 = AdaBoostRegressor(DecisionTreeRegressor(), n_estimators=100, learning_rate=0.1, random_state=1234)
boost_2 = AdaBoostRegressor(DecisionTreeRegressor(), n_estimators=600, learning_rate=0.01, random_state=1234)
##
## Fitting the bagging model
boost_1.fit(X_train, y_train)
boost_2.fit(X_train, y_train)
##
## Predicting to the test subset
boost_1.pred = boost_1.predict(X_test)
boost_2.pred = boost_2.predict(X_test)
##
## Estimating the bias, variance and root mean squared error
boost_1_rmse = "{:.4f}".format(np.sqrt(metrics.mean_squared_error(y_test, boost_1.pred)))
boost_1_bias = "{:.4f}".format(np.mean(y_test - boost_1.pred))
boost_1_var  = "{:.4f}".format(np.var(y_test - boost_1.pred))
print(boost_1_rmse, boost_1_bias, boost_1_var)
boost_2_rmse = "{:.4f}".format(np.sqrt(metrics.mean_squared_error(y_test, boost_2.pred)))
boost_2_bias = "{:.4f}".format(np.mean(y_test - boost_2.pred))
boost_2_var  = "{:.4f}".format(np.var(y_test - boost_2.pred))
print(boost_2_rmse, boost_2_bias, boost_2_var)

In [ ]:
## Plotting the observed vs predicted tree aboveground biomass
plt.scatter(y_test, boost_1.pred, s=60, edgecolor='black', c='red', label='data, 100 estimators \nlearning rate=0.1')
plt.plot(y_test, y_test, c='blue', label='y=x')
plt.xlim([0, 30])
plt.ylim([0, 30])
plt.xlabel("observed age of secondary forests (years)")
plt.ylabel("predicted age of secondary forests (years)")
plt.legend()
plt.show()

In [ ]:
## Plotting the observed vs predicted tree aboveground biomass
plt.scatter(y_test, boost_2.pred, s=60, edgecolor='black', c='red', label='data, 600 estimators \nlearning rate=0.01')
plt.plot(y_test, y_test, c='blue', label='y=x')
plt.xlim([0, 30])
plt.ylim([0, 30])
plt.xlabel("observed age of secondary forests (years)")
plt.ylabel("predicted age of secondary forests (years)")
plt.legend()
plt.show()

In [ ]:
## Calculate variable importance
boost_1_imp = permutation_importance(boost_1, X_train, y_train, n_repeats=100, random_state=1234)
sort_imp = boost_1_imp.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(boost_1_imp.importances[sort_imp].T,
           vert=False, labels=X_train.columns[sort_imp])
ax.set_title("Permutation Importances (train subset)")
fig.tight_layout()
plt.show()

In [ ]:
## Calculate variable importance
boost_2_imp = permutation_importance(boost_2, X_train, y_train, n_repeats=100, random_state=1234)
sort_imp = boost_2_imp.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(boost_2_imp.importances[sort_imp].T,
           vert=False, labels=X_train.columns[sort_imp])
ax.set_title("Permutation Importances (train subset)")
fig.tight_layout()
plt.show()

### And the winner is...
Let's then compare the results from all fitted algorithms: regression trees, bagging and boosting.

You'll need to make a table with the RMSE, bias and variance values that were calculated for all fitted models.

In [ ]:
## First, create a list with the elements you want to display
table_error = [['algorithm', 'RMSE', 'bias', 'variance'],  ## headers
               ['dtr_full', dtr_full_rmse, dtr_full_bias, dtr_full_var], 
               ['dtr_alpha', dtr_alpha_rmse, dtr_alpha_bias, dtr_alpha_var], 
               ['bagg_1', bagg_1_rmse, bagg_1_bias, bagg_1_var],
               ['bagg_2', bagg_2_rmse, bagg_2_bias, bagg_2_var],
               ['boost_1', boost_1_rmse, boost_1_bias, boost_1_var],
               ['boost_2', boost_2_rmse, boost_2_bias, boost_2_var]]
##
## Then, use the tabulate function
print(tabulate(table_error, headers='firstrow', tablefmt='fancy_grid'))